# Overview of DeepBugs replication and comparison with using CNN architecture for Incorrect Binary Operand bug pattern

## using google colab pro high-ram machine with T4 GPU

following readme on github page
* All commands are called from the main directory.
* Python code (most of the implementation) and JavaScript code (for extracting data from .js files) are in the `/python` and `/javascript` directories.
* All data to learn from, e.g., .js files are expected to be in the `/data` directory.
* All data that is generated, e.g., intermediate representations, are written into the main directory. It is recommended to move them into separate directories.
* All generated data files have a timestamp as part of the file name. Below, all files are used with `*`. When running commands multiple times, make sure to use the most recent files.

# Clone github for DeepBugs
I forked the project to my own github in order to corrections to the code.
Certain keras imports are depricated causing errors.

* depricated import

`from tensorflow.python.keras.models import Sequential`

`from tensorflow.python.keras.layers.core import Dense, Dropout`


* correct import statement

`from keras.models import Sequential`

`from keras.layers import Dense, Dropout`

In [1]:
! git clone -b working https://github.com/livingdan/DeepBugs_replication

Cloning into 'DeepBugs_replication'...
remote: Enumerating objects: 507, done.
remote: Counting objects: 100% (135/135), done.
remote: Compressing objects: 100% (51/51), done.
remote: Total 507 (delta 90), reused 120 (delta 84), pack-reused 372
Receiving objects: 100% (507/507), 313.28 MiB | 14.01 MiB/s, done.
Resolving deltas: 100% (284/284), done.
Updating files: 100% (129/129), done.


In [2]:
# move contents of DeepBugs to main directory
! mv DeepBugs_replication/* .

In [3]:
#  install dependencies npm modules acorn, estraverse, walk-sync
# npm is a package manager for the JavaScript programming language
! npm install acorn
! npm install estraverse
! npm install walk-sync

npm WARN saveError ENOENT: no such file or directory, open '/content/package.json'
npm notice created a lockfile as package-lock.json. You should commit this file.
npm WARN enoent ENOENT: no such file or directory, open '/content/package.json'
npm WARN content No description
npm WARN content No repository field.
npm WARN content No README data
npm WARN content No license field.

+ acorn@8.11.3
added 1 package and audited 1 package in 0.982s
found 0 vulnerabilities

npm WARN saveError ENOENT: no such file or directory, open '/content/package.json'
npm WARN enoent ENOENT: no such file or directory, open '/content/package.json'
npm WARN content No description
npm WARN content No repository field.
npm WARN content No README data
npm WARN content No license field.

+ estraverse@5.3.0
added 1 package and audited 2 packages in 0.866s
found 0 vulnerabilities



   ╭────────────────────────────────────────────────────────────────╮
   │                                                            

# 1. Download Training and Testing datasets
* The full corpus can be downloaded [here](http://www.srl.inf.ethz.ch/js150.php) and is expected to be stored in `data/js/programs_all`. It consists of 100.000 training files, listed in `data/js/programs_training.txt`, and 50.000 files for validation, listed in `data/js/programs_eval.txt`.


In [ ]:
! gdown http://files.srl.inf.ethz.ch/data/js_dataset.tar.gz

Downloading...
From: http://files.srl.inf.ethz.ch/data/js_dataset.tar.gz
To: /content/js_dataset.tar.gz
100% 2.63G/2.63G [02:18<00:00, 18.9MB/s]


In [ ]:
! tar -xzf js_dataset.tar.gz
#extracts

In [ ]:
! mkdir data/js/programs_all
! tar -xzf data.tar.gz -C data/js/programs_all
#makes directory and loads the data there

In [ ]:
! mv data/js/programs_all/data/* data/js/programs_all
#move data to the new location

# 2. Embeddings for Identifiers using Word2Vec

The above bug detector rely on a vector representation for identifier names and literals. To use our framework, the easiest is to use the shipped `token_to_vector.json` file. Alternatively, you can learn the embeddings via Word2Vec as follows:

1. Extract identifiers and tokens:

`node javascript/extractFromJS.js tokens --parallel 4 data/js/programs_50_training.txt data/js/programs_50`

  * The command produces `tokens_*.json` files.
  
2. Encode identifiers and literals with context into arrays of numbers (for faster reading during learning):
  
  `python3 python/TokensToTopTokens.py tokens_*.json`
  
  * The arguments are the just created files.
  * The command produces `encoded_tokens_*.json` files and a file `token_to_number_*.json` that assigns a number to each identifier and literal.

3. Learn embeddings for identifiers and literals:
  
  `python3 python/EmbeddingLearnerWord2Vec.py token_to_number_*.json encoded_tokens_*.json`

  * The arguments are the just created files.
  * The command produces a file `token_to_vector_*.json`.

In [ ]:
# Extract identifiers and tokens from all dataset
!node javascript/extractFromJS.js tokens --parallel 4 data/js/programs_all.txt data/js/programs_all

In [ ]:
# move tokens to own directory
!mkdir tokens/
!mv tokens_*.json tokens/

In [ ]:
# Encode identifiers and literals with context into arrays of numbers for training
! python3 python/TokensToTopTokens.py tokens/tokens_*.json

In [ ]:
# move encoded tokens to own directory
!mkdir encoded_tokens/
!mv encoded_tokens_*.json encoded_tokens/
!mv token_to_number_*.json encoded_tokens/

In [ ]:
# Learn embeddings for identifiers and literals training
! python3 python/EmbeddingLearnerWord2Vec.py encoded_tokens/token_to_number_*.json encoded_tokens/encoded_tokens_*.json

In [ ]:
# move token_to_vector file to own directory
!mkdir token_to_vector/
!mv token_to_vector_1*.json token_to_vector/


# 3. Learning a Bug Detector
Creating a bug detector consists of two main steps:

1. Extract positive (i.e., likely correct) and negative (i.e., likely buggy) training examples from code.

2. Train a classifier to distinguish correct from incorrect code examples.

This replication example will address the swapped argument bug detector
* The `SwappedBinOperands` bug detector looks for accidentally swapped Operands.

## 3.1 Extract positive and Negative training examples

`node javascript/extractFromJS.js binOps --parallel 4 data/js/programs_50_training.txt data/js/programs_50`

  * The `--parallel` argument sets the number of processes to run.
  * `programs_50_training.txt` contains files to include (one file per line). To extract data for validation, run the command with `data/js/programs_50_eval.txt`.
  * The last argument is a directory that gets recursively scanned for .js files, considering only files listed in the file provided as the second argument.


### Unzip binOps previously extrated to speed up model

In [4]:
# unzip previously extracted binops to speed up process for training/eval
!unzip binop/training.zip
!unzip binop/eval.zip

Archive:  binop/training.zip
   creating: training/
  inflating: training/binOps_1713539473688.json  
  inflating: training/binOps_1713538745093.json  
  inflating: training/binOps_1713538592712.json  
  inflating: training/binOps_1713539539287.json  
  inflating: training/binOps_1713539564607.json  
  inflating: training/binOps_1713539559688.json  
  inflating: training/binOps_1713539511294.json  
  inflating: training/binOps_1713538414561.json  
  inflating: training/binOps_1713538557802.json  
  inflating: training/binOps_1713538395728.json  
  inflating: training/binOps_1713538836266.json  
  inflating: training/binOps_1713538670156.json  
  inflating: training/binOps_1713538457847.json  
  inflating: training/binOps_1713538699910.json  
  inflating: training/binOps_1713538553088.json  
  inflating: training/binOps_1713538736852.json  
  inflating: training/binOps_1713539110950.json  
  inflating: training/binOps_1713539617852.json  
  inflating: training/binOps_1713538655691.json 

### Full Corpus data set
For google colab the training and eval set it too large to use using high-ram T4 machine.

In [ ]:
# cut the training dataset by 40% as loading full dataset for model causes out of memory error
import random

with open("data/js/programs_training.txt", "r") as f:
    lines = f.readlines()

# Shuffle the list of lines
print(len(lines))
random.shuffle(lines)

# Calculate the number of lines to remove (40%)
num_lines_to_remove = int(len(lines) * 0.4)

# Remove the specified number of lines
for i in range(num_lines_to_remove):
    lines.pop()

print(len(lines))
# Write the remaining lines back to the file
with open("data/js/programs_training.txt", "w") as f:
    f.writelines(lines)

100000
60000


In [ ]:
# Using full dataset training data
! node javascript/extractFromJS.js binOps --parallel 4 data/js/programs_training.txt data/js/programs_all
! mkdir training/
! mv binOps_* training/

Streaming output truncated to the last 5000 lines.

Reading data/js/programs_all/CartoDB/cartodb.js/src/geo/ui/annotation.js

Added binary operations. Total now: 566

Considered binary operations: 1 out of 1 (100%)

Added binary operations. Total now: 2825
Considered binary operations: 22 out of 33 (67%)

Reading data/js/programs_all/CartoDB/cartodb.js/src/geo/ui/search.js

Reading data/js/programs_all/Azure/azure-sdk-for-node/lib/common/lib/services/service.js

Added binary operations. Total now: 2828
Considered binary operations: 3 out of 3 (100%)

Reading data/js/programs_all/AlgoTrader/betfair-sports-api/test/get_market_traded_volume.js

Added binary operations. Total now: 569

Considered binary operations: 3 out of 3 (100%)

Added binary operations. Total now: 489

Considered binary operations: 6 out of 6 (100%)

Reading data/js/programs_all/CartoDB/cartodb.js/src/geo/ui/time_slider.js

Reading data/js/programs_all/AlgoTrader/betfair/lib/emulator.js

Added binary operations. Total

In [ ]:
# cut the eval dataset by 15% as loading full dataset for model causes out of memory error
import random

with open("data/js/programs_eval.txt", "r") as f:
    lines = f.readlines()

# Shuffle the list of lines
print(len(lines))
random.shuffle(lines)

# Calculate the number of lines to remove (15%)
num_lines_to_remove = int(len(lines) * 0.15)

# Remove the specified number of lines
for i in range(num_lines_to_remove):
    lines.pop()

print(len(lines))
# Write the remaining lines back to the file
with open("data/js/programs_eval.txt", "w") as f:
    f.writelines(lines)

50000
42500


In [ ]:
# Using full dataset evaluation data
! node javascript/extractFromJS.js binOps --parallel 4 data/js/programs_eval.txt data/js/programs_all
! mkdir eval/
! mv binOps_* eval/

## 3.2 Train a classifier to identify bugs
1. Train and validate the classifier

`python3 python/BugLearnAndValidate.py --pattern IncorrectBinaryOperand --token_emb token_to_vector.json --type_emb type_to_vector.json --node_emb node_type_to_vector.json --training_data binOps_xx*.json --validation_data binOps_yy*.json`

  * The first argument selects the bug pattern.
  * The next three arguments are vector representations for tokens (here: identifiers and literals), for types, and for AST node types. These files are provided in the repository.
  * The remaining arguments are two lists of .json files. They contain the training and validation data extracted in Step 1.
  * After learning the bug detector, the command measures accurracy and recall w.r.t. seeded bugs and writes a list of potential bugs in the unmodified validation code (see `poss_anomalies.txt`).

### using previously learned by me embeddings token_to_vector_Replication.json to speed up notebook

In [5]:
# Train and validate the classifier
! python3 python/BugLearnAndValidate.py --pattern IncorrectBinaryOperand --token_emb token_to_vector_Replication.json --type_emb type_to_vector.json --node_emb node_type_to_vector.json --training_data training/binOps_*.json --validation_data eval/binOps_*.json

2024-04-22 20:46:11.790464: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-22 20:46:11.790518: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-22 20:46:11.791847: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-22 20:46:11.798928: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-22 20:46:12.762748: W tensorflow/compiler/tf2

In [6]:
# Train and validate the classifier CNN model
! python3 python/BugLearnAndValidateCNN.py --pattern IncorrectBinaryOperand --token_emb token_to_vector_Replication.json --type_emb type_to_vector.json --node_emb node_type_to_vector.json --training_data training/binOps_*.json --validation_data eval/binOps_*.json

2024-04-22 21:13:24.893872: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-22 21:13:24.893917: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-22 21:13:24.913213: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-22 21:13:24.928329: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-22 21:13:26.995115: W tensorflow/compiler/tf2

# 4. Finding Bugs
Finding bugs in one or more source files consists of these two steps:
1. Extract code pieces
2. Use a trained classifier to identify bugs

In [ ]:
# Extract code peices from file directory
#! node javascript/extractFromJS.js calls --files <list of files>
! node javascript/extractFromJS.js binOps --files data/js/programs_50/*.js
! mkdir find_bugs/
! mv binOps_* find_bugs/

Use a trained classifier to identify bugs

`python3 python/BugFind.py --pattern IncorrectBinaryOperand --threshold 0.95 --model some/dir --token_emb token_to_vector.json --type_emb type_to_vector.json --node_emb node_type_to_vector.json --testing_data calls_xx*.json`

  * The first argument selects the bug pattern.
  * 0.95 is the threshold for reporting bugs; higher means fewer warnings of higher certainty.
  * --model sets the directory to load a trained model from.
  * The next three arguments are vector representations for tokens (here: identifiers and literals), for types, and for AST node types. These files are provided in the repository.
  * The remaining argument is a list of .json files. They contain the data extracted in Step 1.
  * The command examines every code piece and writes a list of potential bugs with its probability of being incorrect

In [ ]:
# Use a trained classifier to identify bugs
! python3 python/BugFind.py --pattern IncorrectBinaryOperand --threshold 0.95 --model bug_detection_model_/ --token_emb token_to_vector.json --type_emb type_to_vector.json --node_emb node_type_to_vector.json --testing_data eval/binOps_*.json